## Example Use of David's Finance Module, dfinance
#### See it in action
This notebook shows an example that implements my custom finance module to backtest an (ineffective) algorithmic trading strategy.

In [1]:
import pandas as pd
import yfinance as yf

import dfinance as dfin
from trading_strategies import Strategy_SMA_Crossover

In [2]:
# Initialize backtest object
# Initialize portfolio object
my_back = dfin.Backtest()
my_port = dfin.Portfolio()

In [3]:
# Get cash value - can also set this
my_port.cashvalue

100000.0

In [4]:
# load data from Alpha Vantage csv file
historical_df = pd.read_csv('gme_minute_data.csv', index_col='time')
historical_df.index = pd.to_datetime(historical_df.index)
historical_df.index.name = 'datetime'
historical_df = historical_df.sort_values(by=['datetime'])

# # load data from Yahoo!
# data = yf.download(tickers='AAPL', period='60d', progress=False)
# data.drop(columns=['Adj Close'])
# data.to_csv('aapl_daily_data.csv')
# historical_df = pd.read_csv('aapl_daily_data.csv', index_col='Date')

In [5]:
# Process the data from the historical dataframe
my_back.process_historical_data(historical_df,
                                my_port,
                                Strategy_SMA_Crossover,
                                sma_short=10,
                                sma_long=15,
                                share_num=10,
                                ticker='GME')

2021-02-16 13:48:00 | Short: 49.48 | Long: 49.56 | Short < Long
2021-02-16 13:49:00 | Short: 49.45 | Long: 49.52 | Short < Long
2021-02-16 13:50:00 | Short: 49.41 | Long: 49.48 | Short < Long
2021-02-16 13:51:00 | Short: 49.36 | Long: 49.44 | Short < Long
2021-02-16 13:52:00 | Short: 49.34 | Long: 49.42 | Short < Long
2021-02-16 13:53:00 | Short: 49.31 | Long: 49.39 | Short < Long
2021-02-16 13:54:00 | Short: 49.30 | Long: 49.37 | Short < Long
2021-02-16 13:55:00 | Short: 49.26 | Long: 49.34 | Short < Long
2021-02-16 13:56:00 | Short: 49.24 | Long: 49.32 | Short < Long
2021-02-16 13:57:00 | Short: 49.23 | Long: 49.31 | Short < Long
2021-02-16 13:58:00 | Short: 49.23 | Long: 49.29 | Short < Long
2021-02-16 13:59:00 | Short: 49.23 | Long: 49.28 | Short < Long
2021-02-16 14:00:00 | Short: 49.25 | Long: 49.27 | Short < Long
2021-02-16 14:01:00 | Short: 49.28 | Long: 49.27 | Short > Long

Buying 10 shares of GME for 494.399

2021-02-16 14:02:00 | Short: 49.31 | Long: 49.28 | Short > Long
20

In [6]:
my_port.cashvalue

99994.73999999999

In [7]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,GME,10,49.4399,494.399,buy,2021-02-16 14:01:00
1,GME,-10,49.3101,-493.101,sell,2021-02-16 14:10:00
2,GME,10,49.3301,493.301,buy,2021-02-16 14:16:00
3,GME,-10,49.1100,-491.100,sell,2021-02-16 14:22:00
4,GME,10,49.1601,491.601,buy,2021-02-16 14:33:00
5,GME,-10,49.1900,-491.900,sell,2021-02-16 14:53:00
6,GME,10,49.2060,492.060,buy,2021-02-16 15:04:00
7,GME,-10,49.1800,-491.800,sell,2021-02-16 15:09:00
8,GME,10,49.3300,493.300,buy,2021-02-16 15:14:00
9,GME,-10,49.4300,-494.300,sell,2021-02-16 15:31:00


In [8]:
my_port.summary_of_transactions()

'No positions in the market.'

In [9]:
# Buy one share of GME for $100 - what a great price!
my_port.buy_stock('GME', 1, 100, '2021-01-01 00:00:00')

In [10]:
my_port.save_transactions('my_port.csv')

/Users/davidfreitag/alpaca_proj/dfinance.py:109: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  pd.DataFrame(self.__transactions).to_csv(filepath, index=False)


In [11]:
my_port.load_transactions('my_port.csv')
my_port.summary_of_transactions()

,Ticker,Shares Held,Current Price,Current Value of Shares,Cost to Acquire Shares,Unrealized Gain/Loss,Unrealized Gain/Loss %
0,GME,1,116.650002,116.650002,49.732527,66.917475,57.36603
1,Totals,1,N/A,116.650002,49.732527,66.917475,57.36603
